<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-project/blob/main/BLM6114_project_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sonuçların analizi

In [13]:
!pip install -q transformers pandas torch scikit-learn matplotlib datasets

### LLama ve Gemma sonuçlarının birleştirilmesi

In [5]:
import pandas as pd
pd_llama = pd.read_csv("gsm8k_tr_classified_with_llama_results.csv")
pd_gemma = pd.read_csv("gsm8k_tr_classified_with_gemma_results.csv")

In [6]:
pd_llama.head()

,question,answer,question_type,solution_method,tr_llama_result,is_tr_llama_correct
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,oran-orantı,yüzde hesaplama,Sonuç: 108000,0.0
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,yaş problemi,doğrudan çıkarma,Sonuç: 20,0.0
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,oran-orantı,birim oran,Sonuç: 8,0.0
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,çarpma,doğrudan çarpma,Sonuç: 360,0.0
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,oran-orantı,birim oran,Sonuç: 18,1.0


In [7]:
pd_gemma.head()

,question,answer,question_type,solution_method,gg_gemma_result,is_gg_gemma_correct
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,oran-orantı,yüzde hesaplama,Sonuç: 270 \n,0.0
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,yaş problemi,doğrudan çıkarma,Sonuç: 21 \n,1.0
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,oran-orantı,birim oran,Sonuç: 24 \n,1.0
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,çarpma,doğrudan çarpma,Sonuç: 90 \n,0.0
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,oran-orantı,birim oran,Sonuç: 18 \n,1.0


In [9]:
# Seçilen ortak kolonlar ve model özgü kolonlar
common_cols = ["question", "answer", "question_type", "solution_method"]
llama_cols = common_cols + ["tr_llama_result", "is_tr_llama_correct"]
gemma_cols = common_cols + ["gg_gemma_result", "is_gg_gemma_correct"]

# Ortak kolonlarda birleştirme
# 'inner' birleştirme, her iki DataFrame'de de bulunan satırları korur
merged_df = pd.merge(pd_llama[llama_cols], pd_gemma[gemma_cols], on=common_cols, how='inner')

# Birleştirilmiş DataFrame'i kaydetme
merged_df.to_csv("gsm8k_tr_classified_merged_results.csv", index=False)
print("Birleştirilmiş sonuçlar 'gsm8k_tr_classified_merged_results.csv' dosyasına kaydedildi.")

Birleştirilmiş sonuçlar 'gsm8k_tr_classified_merged_results.csv' dosyasına kaydedildi.


In [10]:
merged_df.head()

,question,answer,question_type,solution_method,tr_llama_result,is_tr_llama_correct,gg_gemma_result,is_gg_gemma_correct
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,oran-orantı,yüzde hesaplama,Sonuç: 108000,0.0,Sonuç: 270 \n,0.0
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,yaş problemi,doğrudan çıkarma,Sonuç: 20,0.0,Sonuç: 21 \n,1.0
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,oran-orantı,birim oran,Sonuç: 8,0.0,Sonuç: 24 \n,1.0
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,çarpma,doğrudan çarpma,Sonuç: 360,0.0,Sonuç: 90 \n,0.0
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,oran-orantı,birim oran,Sonuç: 18,1.0,Sonuç: 18 \n,1.0


## Embedding işlemleri

In [14]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

df = pd.read_csv("gsm8k_tr_classified_merged_results.csv")
print("Veri seti okundu:")
print(df.head())

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-e5-large")
model     = AutoModel.from_pretrained("ytu-ce-cosmos/turkish-e5-large").to(device)

def embed_texts(texts, batch_size=16):
  all_embs = []
  for i in range(0, len(texts), batch_size):
    batch = texts[i : i+batch_size]
    enc = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
      last_hidden = model(**enc).last_hidden_state  # (B, L, D)
      emb = last_hidden.mean(dim=1)                 # mean pooling → (B, D)
    all_embs.append(emb.cpu())
  return torch.cat(all_embs, dim=0).numpy()

for col in ["question", "answer", "question_type", "solution_method"]:
  print(f"Embedding yapılıyor: {col}")
  embs = embed_texts(df[col].astype(str).tolist(), batch_size=16)
  df[f"{col}_emb"] = list(embs) # embedding'i sonuna "_emb" eklenerek yeni kolon olarak ekleniyor.
  print(f"Embedding tamamlandı: {col}")


Veri seti okundu:
                                            question  \
0  Borris tekel bayisi her 6 ayda bir 90 kilogram...   
1  Mel, Katherine'den üç yaş küçük.  Katherine ik...   
2  James 2 ağacındaki tüm meyveleri toplar.  Her ...   
3  Kyle, her biri 15 origami yıldızı alabilen 2 c...   
4  Mark'ın iki evcil hayvanı var: Saniyede 10 adı...   

                                              answer question_type  \
0  Borris şu anda her 6 ayda 90 kilogram üzüm kul...   oran-orantı   
1  Katherine iki düzine yaşına geldiğinde 24 yaşı...  yaş problemi   
2                           James 24 ağaç dikmiştir.   oran-orantı   
3  Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...        çarpma   
4  Kaplumbağanın yarışı berabere bitirmesi için 1...   oran-orantı   

    solution_method tr_llama_result  is_tr_llama_correct gg_gemma_result  \
0   yüzde hesaplama   Sonuç: 108000                  0.0   Sonuç: 270 \n   
1  doğrudan çıkarma       Sonuç: 20                  0.0    Sonuç: 21 \n

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Embedding yapılıyor: question
Embedding tamamlandı: question
Embedding yapılıyor: answer
Embedding tamamlandı: answer
Embedding yapılıyor: question_type
Embedding tamamlandı: question_type
Embedding yapılıyor: solution_method
Embedding tamamlandı: solution_method


In [16]:
df.to_pickle("gsm8k_tr_with_embeddings.pkl")

### Lojistik Regresyon

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_pickle("gsm8k_tr_with_embeddings.pkl")

# Her bir embedding'i (list of floats) (N×d_i) dizisine çevir
Q     = np.array(df['question_emb'].tolist())           # shape=(N, d1)
A     = np.array(df['answer_emb'].tolist())             # shape=(N, d2)
QT    = np.array(df['question_type_emb'].tolist())      # shape=(N, d3)
SM    = np.array(df['solution_method_emb'].tolist())    # shape=(N, d4)

# Hepsini birleştir: X shape = (N, d1+d2+d3+d4)
X = np.concatenate([Q, A, QT, SM], axis=1)

## bildi/bilemedi etiketleri
y_llama = df['is_tr_llama_correct'].values
y_gemma = df['is_gg_gemma_correct'].values

X_train, X_test, y_train_llama, y_test_llama, y_train_gemma, y_test_gemma = train_test_split(
    X, y_llama, y_gemma,
    test_size=0.5,
    random_state=42,
)

In [20]:
# Logistic Regression (gg_gemma)
clf = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
clf.fit(X_train, y_train_gemma)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test_gemma, y_pred))
print(classification_report(y_test_gemma, y_pred))

Accuracy: 0.7179253867151957
              precision    recall  f1-score   support

         0.0       0.86      0.79      0.82      3626
         1.0       0.28      0.38      0.32       770

    accuracy                           0.72      4396
   macro avg       0.57      0.59      0.57      4396
weighted avg       0.76      0.72      0.73      4396



In [21]:
# Logistic Regression (tr_llama)
clf = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
clf.fit(X_train, y_train_llama)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test_llama, y_pred))
print(classification_report(y_test_llama, y_pred))

Accuracy: 0.767515923566879
              precision    recall  f1-score   support

         0.0       0.90      0.83      0.86      3905
         1.0       0.16      0.25      0.20       491

    accuracy                           0.77      4396
   macro avg       0.53      0.54      0.53      4396
weighted avg       0.82      0.77      0.79      4396

